In [ ]:
#imports
import torch
import torch.nn as nn # All Neural Network models
import torch.optim as optim # All Optimization
from torch.utils.data import DataLoader # Easier Dataset Managment 
import torchvision.transforms as transforms # Transformations we can do
import torchvision.datasets as datasets # to import the needed dataset
import torchvision 

In [ ]:
# Set Device & Hyperparameters & Model Loading
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

batch_size = 32
epochs = 20

model = torchvision.models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features # to delete the last layer
model.fc = nn.Linear(num_ftrs, 100) # CIFAR100 has 100 classes
_ = model.to(device)

In [ ]:
# Transformation 
# Define the standard ImageNet transforms
transform = transforms.Compose([
    transforms.Resize(256),              # Resize the image to 256x256 pixels
    transforms.CenterCrop(224),          # Crop the image to 224x224 pixels at the center
    transforms.ToTensor(),               # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], # Normalize pixel values to match the ImageNet distribution
                         std=[0.229, 0.224, 0.225]),
])

In [ ]:
# Load Training Set
trainset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)

# Load Test Set
testset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=4)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
torch.cuda.empty_cache()

In [ ]:
# Training The Model
for epoch in range(epochs):
    running_loss = 0.0
    
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(
            outputs, labels
        )
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        if i % (batch_size*2) == ((batch_size*2) - 1):
            print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

In [ ]:
# Testing The Model
_ = model.eval()  # Set the model to evaluation mode
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(
            outputs.data, 1
        )
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f'Accuracy : {100 * correct / total} %')